In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ajuste import ajustar_precos

In [3]:
# Histórico dos preços de fechamento do HGLG11
hglg11_precos = pd.read_csv('data/hglg11_precos.csv', parse_dates=['data'])
hglg11_precos

,data,preco
0,2011-03-02,104.40
1,2011-03-21,104.40
2,2011-03-28,104.40
3,2011-03-31,104.40
4,2011-04-05,104.40
...,...,...
2779,2022-12-21,159.98
2780,2022-12-22,160.00
2781,2022-12-23,163.00
2782,2022-12-26,163.99


In [4]:
# Histórico de dividendos do HGLG11 em que a data base foi ajustada
# para ser necessariamente em dia com pregão
hglg11_divs = (
    pd.read_csv('data/hglg11_dividendos.csv', parse_dates=['data_com'])
    # Remover as colunas que não serão utilizadas
    .drop(columns=['tipo', 'data_pagamento'])
)
hglg11_divs

,data_com,valor_div
0,2011-03-31,0.72
1,2011-04-29,0.75
2,2011-05-31,0.79
3,2011-06-30,0.80
4,2011-07-29,0.81
...,...,...
136,2022-07-29,1.10
137,2022-08-31,1.10
138,2022-09-30,1.10
139,2022-10-31,1.10


In [5]:
hglg11_precos = ajustar_precos(hglg11_precos, hglg11_divs)
hglg11_precos

,data,preco,preco_aj,valor_div
0,2011-03-02,104.40,37.450552,NaN
1,2011-03-21,104.40,37.450552,NaN
2,2011-03-28,104.40,37.450552,NaN
3,2011-03-31,104.40,37.450552,0.72
4,2011-04-05,104.40,37.710625,NaN
...,...,...,...,...
2779,2022-12-21,159.98,159.980000,NaN
2780,2022-12-22,160.00,160.000000,NaN
2781,2022-12-23,163.00,163.000000,NaN
2782,2022-12-26,163.99,163.990000,NaN


In [6]:
# Calcular o dividend_yield
hglg11_precos["dividend_yield(%)"] = 100 * hglg11_precos["valor_div"] / hglg11_precos["preco"]
hglg11_precos.query("valor_div > 0")

,data,preco,preco_aj,valor_div,dividend_yield(%)
3,2011-03-31,104.40,37.450552,0.72,0.689655
13,2011-04-27,104.40,37.710625,0.75,0.718391
27,2011-05-31,104.30,37.947112,0.79,0.757430
39,2011-06-30,106.30,38.969935,0.80,0.752587
48,2011-07-21,109.00,40.262777,0.81,0.743119
...,...,...,...,...,...
2680,2022-07-29,168.64,163.207107,1.10,0.652277
2703,2022-08-31,172.49,168.029090,1.10,0.637718
2724,2022-09-30,171.50,168.136933,1.10,0.641399
2744,2022-10-31,165.39,163.193471,1.10,0.665095


In [7]:
# Última distribuição de dividendos
hglg11_precos.query('data >= "2022-11-27"').head()

,data,preco,preco_aj,valor_div,dividend_yield(%)
2762,2022-11-28,162.90,161.812748,NaN,NaN
2763,2022-11-29,163.50,162.408743,NaN,NaN
2764,2022-11-30,164.81,163.710000,1.1,0.667435
2765,2022-12-01,162.71,162.710000,NaN,NaN
2766,2022-12-02,162.80,162.800000,NaN,NaN


In [8]:
# Comparar os preços com e sem ajuste do HGLG11 com o Plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco"],
        mode="lines",
        line=dict(color="#97c9cc"),
        name="Preço sem ajuste",        
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj"],
        mode="lines",
        line=dict(color="#02878e"),
        name="Preço com ajuste",
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento do HGLG11 <br>(com e sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_1', auto_open=True)

In [9]:
# Valorização do HGLG11 com e sem ajuste de dividendos
valorizacao_sa = (hglg11_precos["preco"].iloc[-1] / hglg11_precos["preco"].iloc[0] - 1) * 100
valorizacao_ca = (hglg11_precos["preco_aj"].iloc[-1] / hglg11_precos["preco_aj"].iloc[0] - 1) * 100
print(f"Valorização do HGLG11 com ajuste de dividendos: {valorizacao_sa:.2f}%")
print(f"Valorização do HGLG11 sem ajuste de dividendos: {valorizacao_ca:.2f}%")

Valorização do HGLG11 com ajuste de dividendos: 57.32%
Valorização do HGLG11 sem ajuste de dividendos: 338.55%


In [10]:
# Carregar os dados de preços e dividendos do TAEE11
taee11_precos = (
    pd.read_csv('data/taee11_precos.csv', parse_dates=['Date'])
    .rename(columns={'Close': 'preco', 'Date': 'data'})
    # Concidir com o período de negociação do HGLG11
    .query('data >= "2011-03-02"')
    # Selecionar apenas as colunas de interesse
    [['data', 'preco']]
    .reset_index(drop=True)
)
taee11_precos

,data,preco
0,2011-03-02,10.370000
1,2011-03-03,10.370000
2,2011-03-04,10.370000
3,2011-03-09,10.370000
4,2011-03-10,10.666666
...,...,...
2930,2022-12-21,33.770000
2931,2022-12-22,34.669998
2932,2022-12-23,34.810001
2933,2022-12-26,34.630001


In [11]:
# Carregar os dados de dividendos do TAEE11
taee11_divs = (
    pd.read_csv('data/taee11_dividendos.csv', parse_dates=['Date'])
    .rename(columns={'Dividends': 'valor_div', 'Date': 'data_com'})
)
taee11_divs[-5:]

,data_com,valor_div
39,2021-05-19,1.354363
40,2021-12-07,1.518146
41,2022-05-10,1.896324
42,2022-08-16,1.470929
43,2022-11-16,0.131087


In [12]:
# Total de dividendos pagos pelo TAEE11
taee11_divs.valor_div.sum()

30.13772

In [13]:
# Total de dividendos pagos pelo TAEE11 em 2022
taee11_divs.query('data_com.dt.year == 2022').valor_div.sum()

3.49834

In [14]:
# Ajustar os preços de fechamento da TAEE11
taee11_precos = ajustar_precos(taee11_precos, taee11_divs)
taee11_precos

,data,preco,preco_aj,valor_div
0,2011-03-02,10.370000,3.093232,NaN
1,2011-03-03,10.370000,3.093232,NaN
2,2011-03-04,10.370000,3.093232,NaN
3,2011-03-09,10.370000,3.093232,NaN
4,2011-03-10,10.666666,3.181723,NaN
...,...,...,...,...
2930,2022-12-21,33.770000,33.770000,NaN
2931,2022-12-22,34.669998,34.669998,NaN
2932,2022-12-23,34.810001,34.810001,NaN
2933,2022-12-26,34.630001,34.630001,NaN


In [15]:
# Calcular o dividend_yield
taee11_precos["dividend_yield(%)"] = 100 * taee11_precos["valor_div"] / taee11_precos["preco"]
# Dividend yield mais recentes
taee11_precos.query("valor_div > 0").tail()

,data,preco,preco_aj,valor_div,dividend_yield(%)
2532,2021-05-19,39.250000,33.281457,1.354363,3.450606
2670,2021-12-07,36.150002,31.748371,1.518146,4.199574
2774,2022-05-10,41.099998,37.677969,1.896324,4.613927
2843,2022-08-16,41.400002,39.788822,1.470929,3.552968
2905,2022-11-16,37.320000,37.188913,0.131087,0.351251


In [16]:
# Adicionar uma coluna com o preço normalizado em taee11_precos e hglg11_precos
taee11_precos["preco_normalizado"] = 100 * taee11_precos["preco"] / taee11_precos["preco"].iloc[0]
hglg11_precos["preco_normalizado"] = 100 * hglg11_precos["preco"] / hglg11_precos["preco"].iloc[0]

In [18]:
# Plotar os preços da TAEE11 e HGLG11 sem ajustes
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=taee11_precos["data"],
        y=taee11_precos["preco_normalizado"],
        mode="lines",
        name="TAEE11",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
    )
)
# Adicionar os pontos de dividendos da TAEE11 no próprio gráfico de preços
fig.add_trace(
    go.Scatter(
        x=taee11_precos.query('valor_div > 0')["data"],
        y=taee11_precos.query('valor_div > 0')["preco_normalizado"],
        mode="markers",
        name="Dividendos da TAEE11",
        marker=dict(
            color="black",
            size=(30 * taee11_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
# Adicionar os pontos de dividendos da HGLG11 no próprio gráfico de preços, sendo que o tamanho dos pontos é proporcional ao valor do dividendo
fig.add_trace(
    go.Scatter(
        x=hglg11_precos.query('valor_div > 0')["data"],
        y=hglg11_precos.query('valor_div > 0')["preco_normalizado"],
        mode="markers",
        name="Dividendos do HGLG11",
        marker=dict(
            color="black",
            size=(30 * hglg11_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (normalizado para R$ 100 em 2011-03-02 e sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()
# py.plot(fig, filename = 'hglg_petr_2', auto_open=True)

In [17]:
taee11_precos.query('valor_div > 0')

,data,preco,preco_aj,valor_div,dividend_yield(%),preco_normalizado
37,2011-04-28,10.500000,3.132009,1.265340,12.050857,101.253616
288,2012-04-30,21.830000,7.403820,0.813163,3.724979,210.511090
447,2012-12-19,21.490000,7.570506,0.464443,2.161205,207.232401
534,2013-05-02,21.750000,7.831351,1.042920,4.795034,209.739634
691,2013-12-13,18.959999,7.170610,0.551526,2.908893,182.835092
782,2014-05-02,19.340000,7.533466,1.200777,6.208775,186.499518
933,2014-12-05,18.750000,7.787130,0.571845,3.049840,180.810029
1024,2015-04-23,20.049999,8.588987,0.739500,3.688279,193.346181
1126,2015-09-18,18.799999,8.361925,0.899979,4.787123,181.292179
1189,2015-12-21,16.150000,7.544410,0.320006,1.981461,155.737705


In [23]:
# Adicionar uma coluna com o preço ajustado e normalizado em taee11_precos e hglg11_precos
taee11_precos["preco_aj_normalizado"] = 100 * taee11_precos["preco_aj"] / taee11_precos["preco_aj"].iloc[0]
hglg11_precos["preco_aj_normalizado"] = 100 * hglg11_precos["preco_aj"] / hglg11_precos["preco_aj"].iloc[0]

In [21]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=taee11_precos["data"],
        y=taee11_precos["preco_aj"],
        mode="lines",
        name="TAEE11",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
        
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (com ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [24]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=taee11_precos["data"],
        y=taee11_precos["preco_aj_normalizado"],
        mode="lines",
        name="TAEE11",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=hglg11_precos["data"],
        y=hglg11_precos["preco_aj_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
        
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (normalizado para R$ 100 em 2011-03-02 e com ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()